In [10]:
import pandas as pd 
import seaborn as sns
import numpy as np
import  matplotlib.pyplot as plt

In [11]:
# 导入数据
input_data = pd.read_excel(r'/Users/dylan/Github repo/DataBase/signal_process/benchmark_result-30_5.xlsx',sheet_name = 'Sheet1')
signal_input = pd.read_excel(r'/Users/dylan/Github repo/DataBase/signal_process/benchmark_result-30_5.xlsx',sheet_name = 'Sheet3')

signal_input_roll = signal_input[signal_input['model'] == 'benchmark']

In [12]:
# 数据清理
signal_input_roll.drop(labels = 'Unnamed: 0',axis = 1, inplace = True)
signal_input_roll.reset_index(drop = True)
signal_input.drop(labels = 'Unnamed: 0',axis = 1, inplace = True)
signal_input.reset_index(drop = True)


/Users/dylan/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,start_date,end_date,model,annual_return_sp,sharpe_ratio_sp,win_rate_sp,pc_ratio_sp,sp_rank
0,2018-05-01,2018-06-12,2日开盘价持续下跌-1120-bagc,3.41,3.27,0.41,1.87,0.656716
1,2018-05-01,2018-06-12,2日开盘价持续下跌-1120-dtc,3.41,3.27,0.41,1.87,0.656716
2,2018-05-01,2018-06-12,2日开盘价持续下跌-1120-knnc,-0.28,-0.35,0.25,1.05,0.074627
3,2018-05-01,2018-06-12,2日开盘价持续下跌-1120-lr,3.41,3.27,0.41,1.87,0.656716
4,2018-05-01,2018-06-12,2日开盘价持续下跌-1120-rfc,2.02,1.43,0.40,1.03,0.231343
...,...,...,...,...,...,...,...,...
9643,2021-04-08,2021-05-24,2日开盘价持续下跌-1230-knnc,-1.67,-1.51,0.67,1.78,0.626866
9644,2021-04-08,2021-05-24,2日开盘价持续下跌-1230-lr,-2.18,-1.34,0.00,1.55,0.723881
9645,2021-04-08,2021-05-24,2日开盘价持续下跌-1230-rfc,-6.01,-4.67,0.50,1.33,0.231343
9646,2021-04-08,2021-05-24,2日开盘价持续下跌-1230-xgb,-4.62,-3.12,1.00,1.59,0.462687


In [13]:
# 函数定义
def get_value(m):
    value = m['sharpe_ratio_sp'] - m[m['model'] == 'benchmark']['sharpe_ratio_sp'].iat[0] # 返回一个序列
    return value

def get_lable(n):
    if n >= 0:
        return True
    else: 
        return False

In [16]:
# 进行无加权平均回溯
# 得到整个验证期的model_list
list_ = list(signal_input_roll.rolling(window = 10,min_periods = 10)['start_date'])[10:]    
model_list = pd.DataFrame()
list1 = []
for i in range(len(list_)):
    list1 = list1 + list(list_[i])
    x = signal_input.set_index('start_date',inplace = False).loc[list1]
    x['value'] = list(x.groupby('end_date').apply(get_value)) # 转为list
    x['lable'] = list(x['value'].apply(get_lable))
    x.loc[:,'value'] = x.transform({'value' : lambda x : np.square(x) if x >= 0 else (-np.square(x))})
    signal_input_value = x
    # input_data.groupby(['model','lable']).sum().apply(get_loss)
    input_filter = signal_input_value.groupby(['model','lable']).agg({'value' : 'sum'})
    input_filter.drop(input_filter.tail(1).index,inplace=True)
    input_filter = input_filter.unstack(fill_value = 0).loc[:,'value']
    input_filter.loc[:,'loss'] = input_filter[True] - input_filter[False]
    model_filter = input_filter.nlargest(10,'loss',keep = 'first')
    # model_list  = list(input_filter['loss'].nlargest(10).index.get_level_values(0).drop_duplicates())
    model_list = model_list.append(model_filter)
model_list.head(50)
# 更改为链式写法



lable,False,True,loss
model,,,
2日开盘价持续下跌-1131-rfc,0.0000,146.8006,146.8006
2日开盘价持续下跌-1122-bagc,-128.1187,0.5266,128.6453
2日开盘价持续下跌-1131-dtc,0.0000,126.5903,126.5903
2日开盘价持续下跌-1130-xgb,0.0000,108.7332,108.7332
2日开盘价持续下跌-1121-dtc,0.0000,102.8724,102.8724
2日开盘价持续下跌-1121-rfc,0.0000,102.8724,102.8724
2日开盘价持续下跌-1121-xgb,0.0000,102.8724,102.8724
2日开盘价持续下跌-1132-knnc,0.0000,99.2188,99.2188
2日开盘价持续下跌-1120-xgb,0.0000,96.4601,96.4601


In [ ]:
# 统计频次
print(len(list1))
pd.Series(model_list.index).value_counts()

In [18]:
# 进行指数加权平均回溯

list_ = list(signal_input_roll.rolling(window = 10,min_periods = 10)['start_date'])[10:]    
model_list = pd.DataFrame()
list1 = []
all_model = signal_input.model.drop_duplicates()
for i in range(len(list_)):
    list1 = list1 + list(list_[i])
    x = signal_input.set_index('start_date',inplace = False).loc[list1]
    x['value'] = list(x.groupby('end_date').apply(get_value)) # 转为list
    x['lable'] = list(x['value'].apply(get_lable))
    x.loc[:,'value'] = x.transform({'value' : lambda x : np.square(x) if x >= 0 else (-np.square(x))})
    # input_data.groupby(['model','lable']).sum().apply(get_loss)
    # 指数加权平均
    # input_filter = x.groupby(['model','lable']).emw(alpha = 0.9, adjust = False)
    input_filter = pd.DataFrame()
    for model in all_model:
        x[x.model == model].loc[:,'value'] = x[x.model == model].value.ewm(alpha = 0.6, adjust = False).mean()
        input_filter = input_filter.append(x[x.model == model])
    input_filter = input_filter.groupby(['model','lable']).agg({'value' : 'sum'})
    input_filter.drop(input_filter.tail(1).index,inplace=True)
    input_filter = input_filter.unstack(fill_value = 0).loc[:,'value']
    input_filter.loc[:,'loss'] = input_filter[True] - input_filter[False]
    model_filter = input_filter.nlargest(10,'loss',keep = 'first')
    #model_list  = list(input_filter['loss'].nlargest(10).index.get_level_values(0).drop_duplicates())
    model_list = model_list.append(model_filter)
model_list